# Pull code from https://github.com/flibustier/pokemon-tcg-pocket-database

In [1]:
import pandas as pd
import json

In [15]:
# Read JSON cards data from file
with open("/Users/72561/Documents/Personal/Trade_Pokemon/pokemon-tcg-pocket-database/dist/cards.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Convert JSON to DataFrame. 
# json_normalize automatically flattens nested objects like "label".
df = pd.json_normalize(data)

df.head()

,set,number,rarity,rarityCode,imageName,packs,label.slug,label.eng
0,A1,1,Common,C,cPK_10_000010_00_FUSHIGIDANE_C.webp,[Mewtwo],bulbasaur,Bulbasaur
1,A1,2,Uncommon,U,cPK_10_000020_00_FUSHIGISOU_U.webp,[Mewtwo],ivysaur,Ivysaur
2,A1,3,Rare,R,cPK_10_000030_00_FUSHIGIBANA_R.webp,[Mewtwo],venusaur,Venusaur
3,A1,4,Double Rare,RR,cPK_10_000040_00_FUSHIGIBANAex_RR.webp,[Mewtwo],venusaur-ex,Venusaur ex
4,A1,5,Common,C,cPK_10_000050_00_CATERPIE_C.webp,[Pikachu],caterpie,Caterpie


In [16]:
mask = df[['set', 'rarity', 'label.slug']].duplicated(keep=False)

df.loc[mask, 'label.eng'] = (
    df.loc[mask, 'label.eng'] + " " + df.loc[mask, 'number'].astype(str)
)


In [17]:
df[df[['set', 'rarity','label.slug']].duplicated(keep=False)]

,set,number,rarity,rarityCode,imageName,packs,label.slug,label.eng
165,A1,166,Common,C,cPK_10_001660_00_NIDORAN♀_C.webp,[Pikachu],nidoran,Nidoran♀ 166
168,A1,169,Common,C,cPK_10_001690_00_NIDORAN♂_C.webp,[Pikachu],nidoran,Nidoran♂ 169
205,A1,206,Common,C,cPK_10_002060_00_EIEVUI_C.webp,[Charizard],eevee,Eevee 206
206,A1,207,Common,C,cPK_10_002060_01_EIEVUI_C.webp,[Mewtwo],eevee,Eevee 207
207,A1,208,Common,C,cPK_10_002060_02_EIEVUI_C.webp,[Pikachu],eevee,Eevee 208
387,A2,16,Common,C,cPK_10_002970_00_MINOMADAMKUSAKINOMINO_C.webp,"[Dialga, Palkia]",wormadam,Wormadam 16
461,A2,90,Common,C,cPK_10_003710_00_MINOMADAMSUNACHINOMINO_C.webp,[Dialga],wormadam,Wormadam 90
486,A2,115,Common,C,cPK_10_003960_00_MINOMADAMGOMINOMINO_C.webp,[Palkia],wormadam,Wormadam 115
585,A2A,7,Uncommon,U,cPK_10_004310_00_CHERRIM_U.webp,[Arceus],cherrim,Cherrim 7
586,A2A,8,Uncommon,U,cPK_10_004320_00_CHERRIM_U.webp,[Arceus],cherrim,Cherrim 8


In [24]:
# import requests
# from io import BytesIO
# from PIL import Image
# from IPython.display import display

# # Example base URL: adjust as needed
# BASE_URL = "https://assets.pokemon-zone.com/game-assets/CardPreviews"

# def show_images_from_df(df):
#     """Display images inline for each row in the DataFrame."""
#     for idx, row in df.iterrows():
#         # Build the full image URL
#         image_url = f"{BASE_URL}/{row['imageName']}"
        
#         # Download the image
#         response = requests.get(image_url)
        
#         # If successful, display inline
#         if response.ok:
#             # Convert bytes into an image
#             image_data = BytesIO(response.content)
#             pil_image = Image.open(image_data)
#             display(pil_image)
#         else:
#             print(f"Failed to fetch image for {row['label.eng']} at {image_url}: HTTP {response.status_code}")


In [25]:
# show_images_from_df(df.head())

In [19]:
# Read JSON expansion data from file
with open("/Users/72561/Documents/Personal/Trade_Pokemon/pokemon-tcg-pocket-database/dist/sets.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Convert JSON to DataFrame. 
# json_normalize automatically flattens nested objects like "label".
df2 = pd.json_normalize(data)

df2.head(10)

,code,releaseDate,count,packs,label.en
0,A1,2024-10-30,286.0,"[Charizard, Mewtwo, Pikachu]",Genetic Apex
1,A1A,2024-12-17,85.0,[Mew],Mythical Island
2,A2,2025-01-29,207.0,"[Dialga, Palkia]",Space-Time Smackdown
3,A2A,2025-02-28,96.0,[Arceus],Triumphant Light
4,A2B,2025-03-27,111.0,[Shining],Shining Revelry
5,A3,2025-04-30,239.0,"[Lunala, Solgaleo]",Celestial Guardians
6,A3A,2025-05-29,103.0,[Extradimensional],Extradimensional Crisis
7,PROMO-A,2024-10-30,NaN,"[Vol. 1, Vol. 2, Vol. 3, Vol. 4, Vol. 5, Vol. ...",Promo A


In [20]:
#join two dfs on set - code
df_complete = df.merge(df2, left_on='set', right_on='code', suffixes=('', '_expansion'))
df_complete.head()

,set,number,rarity,rarityCode,imageName,packs,label.slug,label.eng,code,releaseDate,count,packs_expansion,label.en
0,A1,1,Common,C,cPK_10_000010_00_FUSHIGIDANE_C.webp,[Mewtwo],bulbasaur,Bulbasaur,A1,2024-10-30,286.0,"[Charizard, Mewtwo, Pikachu]",Genetic Apex
1,A1,2,Uncommon,U,cPK_10_000020_00_FUSHIGISOU_U.webp,[Mewtwo],ivysaur,Ivysaur,A1,2024-10-30,286.0,"[Charizard, Mewtwo, Pikachu]",Genetic Apex
2,A1,3,Rare,R,cPK_10_000030_00_FUSHIGIBANA_R.webp,[Mewtwo],venusaur,Venusaur,A1,2024-10-30,286.0,"[Charizard, Mewtwo, Pikachu]",Genetic Apex
3,A1,4,Double Rare,RR,cPK_10_000040_00_FUSHIGIBANAex_RR.webp,[Mewtwo],venusaur-ex,Venusaur ex,A1,2024-10-30,286.0,"[Charizard, Mewtwo, Pikachu]",Genetic Apex
4,A1,5,Common,C,cPK_10_000050_00_CATERPIE_C.webp,[Pikachu],caterpie,Caterpie,A1,2024-10-30,286.0,"[Charizard, Mewtwo, Pikachu]",Genetic Apex


In [21]:
df_final = df_complete[['label.en', 'label.eng', 'rarityCode', 'imageName']]
df_final.columns = ['Espansione', 'Nome', 'Rarità', 'Immagine']
df_final.head()

,Espansione,Nome,Rarità,Immagine
0,Genetic Apex,Bulbasaur,C,cPK_10_000010_00_FUSHIGIDANE_C.webp
1,Genetic Apex,Ivysaur,U,cPK_10_000020_00_FUSHIGISOU_U.webp
2,Genetic Apex,Venusaur,R,cPK_10_000030_00_FUSHIGIBANA_R.webp
3,Genetic Apex,Venusaur ex,RR,cPK_10_000040_00_FUSHIGIBANAex_RR.webp
4,Genetic Apex,Caterpie,C,cPK_10_000050_00_CATERPIE_C.webp


In [22]:
df_final.Espansione.unique()

array(['Genetic Apex', 'Mythical Island', 'Space-Time Smackdown',
       'Triumphant Light', 'Shining Revelry', 'Celestial Guardians',
       'Extradimensional Crisis', 'Promo A'], dtype=object)

In [23]:
# Filter out cards that cannot be traded
df_final = df_final[df_final['Espansione'] != 'Promo A']
df_final = df_final[~df_final['Rarità'].isin(['SR', 'SAR', 'IM', 'UR', 'SR', 'S', 'SSR'])]

In [24]:
df_final.Rarità.unique()

array(['C', 'U', 'R', 'RR', 'AR'], dtype=object)

In [25]:
#print(df_final['Rarità'].unique())

mappa_simboli_rarità = {
    'C':  '♦',
    'U':  '♦♦',
    'R':  '♦♦♦',
    'RR': '♦♦♦♦',
    'AR': '★'
}

df_final['Rarità'] = df_final['Rarità'].map(mappa_simboli_rarità)

#print(df_final['Rarità'].unique())

In [26]:
df_final.to_csv('/Users/72561/Documents/Personal/Trade_Pokemon/pokemon-trade/static/files/Anagrafica_Pokemon.csv', index=False, encoding='utf-8-sig', sep=';')